In [12]:
import pandas as pd
import tensorflow as tf

teams = {}
teams_stats = {}
for i in range(10, 17):
    df = pd.read_csv("./archive-3/Datasets/20" + str(i) + "-" + str(i + 1) + ".csv")
    cur_teams = set(df["HomeTeam"].unique())
    teams[i] = cur_teams
    cur_teams_stats = {}
    for team in cur_teams:
        cur_teams_stats[team] = [0, 0, 0]
    for index, row in df.iterrows():
        cur_teams_stats[row["HomeTeam"]][1] += row["FTHG"]
        cur_teams_stats[row["HomeTeam"]][2] += row["FTAG"]
        cur_teams_stats[row["AwayTeam"]][1] += row["FTAG"]
        cur_teams_stats[row["AwayTeam"]][2] += row["FTHG"]
        if row["FTHG"] > row["FTAG"]:
            cur_teams_stats[row["HomeTeam"]][0] += 3
        elif row["FTHG"] == row["FTAG"]:
            cur_teams_stats[row["HomeTeam"]][0] += 1
            cur_teams_stats[row["AwayTeam"]][0] += 1
        else:
            cur_teams_stats[row["AwayTeam"]][0] += 3
    teams_stats[i] = cur_teams_stats

In [13]:
df_train_test = pd.DataFrame(columns=["Home Team", "Away Team",
                                      "Points Obtained Last Season", "Points Obtained by Opponent Last Season",
                                      "Goals Scored Last Season", "Goals Conceded by Opponent Last Season",
                                      "Goals Scored in Game"])

In [14]:
for i in range(11, 18):
    df = pd.read_csv("./archive-3/Datasets/20" + str(i) + "-" + str(i + 1) + ".csv")
    print(i, len(df))
    for index, row in df.iterrows():
        row1 = [1, 0, 38, 38, 38, 38, row["FTHG"]]
        if row1[-1] > 5:
            row1[-1] = 5
        row2 = [0, 1, 38, 38, 38, 38, row["FTAG"]]
        if row2[-1] > 5:
            row2[-1] = 5
        if row["HomeTeam"] in teams[i - 1]:
            row1[2] = teams_stats[i - 1][row["HomeTeam"]][0]
            row1[4] = teams_stats[i - 1][row["HomeTeam"]][1]
            row2[3] = teams_stats[i - 1][row["HomeTeam"]][0]
            row2[5] = teams_stats[i - 1][row["HomeTeam"]][2]
        if row["AwayTeam"] in teams[i - 1]:
            row1[3] = teams_stats[i - 1][row["AwayTeam"]][0]
            row1[5] = teams_stats[i - 1][row["AwayTeam"]][2]
            row2[2] = teams_stats[i - 1][row["AwayTeam"]][0]
            row2[4] = teams_stats[i - 1][row["AwayTeam"]][1]
        df_train_test.loc[-1] = row1  # adding a row
        df_train_test.index = df_train_test.index + 1  # shifting index
        df_train_test = df_train_test.sort_index()
        df_train_test.loc[-1] = row2  # adding a row
        df_train_test.index = df_train_test.index + 1  # shifting index
        df_train_test = df_train_test.sort_index()         

    
    
    

11 380
12 380
13 380
14 380
15 380
16 380
17 380


In [17]:
df_train = df_train_test[:-760]
print(len(df_train))
print(len(df_train_test))

4560
5320


In [27]:
from tensorflow.keras import layers, models

model = models.Sequential()
model.add(layers.Dense(64, input_dim=6, activation='relu'))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='linear'))  # Output layer with 4 neurons for 'a', 'b', 'c', 'd'

# Compile the model
model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_squared_error'])

In [ ]:
model.fit(df_train[["Home Team", "Away Team",
                    "Points Obtained Last Season", "Points Obtained by Opponent Last Season",
                    "Goals Scored Last Season", "Goals Conceded by Opponent Last Season"]],
          df_train[["Goals Scored in Game"]], epochs=30, validation_split=0.2)

Epoch 1/30
114/114 [==============================] - 1s 6ms/step - loss: 0.9928 - mean_squared_error: 1.5580 - val_loss: 0.9208 - val_mean_squared_error: 1.4434
Epoch 2/30
114/114 [==============================] - 1s 5ms/step - loss: 0.9423 - mean_squared_error: 1.4244 - val_loss: 0.9870 - val_mean_squared_error: 1.4388
Epoch 3/30
114/114 [==============================] - 1s 5ms/step - loss: 0.9836 - mean_squared_error: 1.5474 - val_loss: 1.0690 - val_mean_squared_error: 1.9891
Epoch 4/30
114/114 [==============================] - 1s 5ms/step - loss: 0.9663 - mean_squared_error: 1.4995 - val_loss: 0.9127 - val_mean_squared_error: 1.3221
Epoch 5/30
114/114 [==============================] - 1s 5ms/step - loss: 0.9651 - mean_squared_error: 1.5111 - val_loss: 0.9058 - val_mean_squared_error: 1.2976
Epoch 6/30
114/114 [==============================] - 1s 5ms/step - loss: 0.9418 - mean_squared_error: 1.4394 - val_loss: 0.9138 - val_mean_squared_error: 1.4178
Epoch 7/30
114/114 [========

In [25]:
df_test = df_train_test[-760:]
print(len(df_test))
preds = model.predict(df_test[["Home Team", "Away Team",
                               "Points Obtained Last Season", "Points Obtained by Opponent Last Season",
                               "Goals Scored Last Season", "Goals Conceded by Opponent Last Season"]])

760
24/24 [==============================] - 0s 2ms/step


In [26]:
model.evaluate(df_test[["Home Team", "Away Team",
                               "Points Obtained Last Season", "Points Obtained by Opponent Last Season",
                               "Goals Scored Last Season", "Goals Conceded by Opponent Last Season"]],
              df_test[["Goals Scored in Game"]])

24/24 [==============================] - 0s 4ms/step - loss: 0.9483 - mean_squared_error: 1.4194


[0.9482631683349609, 1.4194339513778687]